# How to train your DragoNN: 
## Exploring convolutional neural network (CNN) architectures for simulated genomic data as well as ENCODE TF ChIP-seq datasets. 

This tutorial will take approximately 2 hours if executed on a GPU. 

**TODO**: Fix

## Outline<a name='outline'>
<ol>
    <li><a href=#9>Training on real data: SPI1 ChIP-seq and experimental bQTL data</a></li>
    <li><a href=#10>Genomewide regression labels for SPI1 TF ChIP-seq</a></li>
    <li><a href=#12>Genome-wide regression for SPI1</a></li> 
    <li><a href=#13>Genome-wide interpretation of true positive predictions in SPI1</a></li>
    <li><a href=#14>Applications for SPI1 bQTL dataset</a></li>
    <ol>
        <li><a href=#a>Read in and annotate SPI1 bQTL dataset</a></li>
        <li><a href=#b>Obatin DNN predictions for POST and ALT alleles within the bQTL dataset</a></li>
        <li><a href=#c>Compare model predictions on POST and ALT alleles </a></li>
        <li><a href=#d>bQTL dataset motif scan with HOCOMOCO SPI1 motif </a></li>
        <li><a href=#e>bQTL interpretation summary: deep learning models vs motif scan</a></li>
    </ol>
    <li><a href=#15>Deep learning models are able to identify low-affinity TF-binding sites missed by motif scanning.</a></li>
</ol>
    

## How to use this tutorial<a name='1'>
<a href=#outline>Home</a>

This tutorial utilizes a Google Colaboratory Notebook - an interactive computational enviroment that combines live code, visualizations, and explanatory text. The notebook is organized into a series of cells. 

The first thing we do is set our Runtime to use Python3 and GPU. 

![ChangeRuntime](https://github.com/kundajelab/dragonn/blob/master/tutorials/tutorial_images/ChangeRuntime.png?raw=true)

![RuntimeType.png](https://github.com/kundajelab/dragonn/blob/master/tutorials/tutorial_images/RuntimeType.png?raw=1)

Now that we set our Runtime, we can execute the cells in the notebook. You can execute the cells one at a time by clicking inside of them and pressing SHIFT+enter. Alternatively, you can run all the cells by clicking the "Run All" button, as demonstrated below. 

![RunAllColab](https://github.com/kundajelab/dragonn/blob/master/tutorials/tutorial_images/RunAllCollab.png?raw=1)


You can run the next cell by cliking the play button:

![RunCellArrow](https://github.com/kundajelab/dragonn/blob/master/tutorials/tutorial_images/RunCellArrow.png?raw=1)

Half of the cells in this tutorial contain code, the other half contain visualizations and explanatory text. Code, visualizations, and text in cells can be modified - you are encouraged to modify the code as you advance through the tutorial. You can inspect the implementation of a function used in a cell by following these steps:

![inspecting code](https://github.com/kundajelab/dragonn/blob/master/tutorials/tutorial_images/inspecting_code.png?raw=1)


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# if running locally
# import sys
# sys.path.append('../../')

In [ ]:
#uncomment the lines below if you are running this tutorial from Google Colab 
# RESTART NOTEBOOK AFTER RUNNING THIS
! pip install git+https://github.com/kundajelab/dragonn.git@cshl
! pip install git+https://github.com/kundajelab/shap.git

In [ ]:
!pip show tensorflow
!pip show dragonn 

In [ ]:
# Making sure our results are reproducible
from numpy.random import seed
seed(1234)
from tensorflow.random import set_seed
set_seed(1234)
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

In [ ]:
#load dragonn tutorial utilities 
from matplotlib import pyplot as plt
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
#uncomment the lines below if you are running this tutorial from Google Colab 
# !pip uninstall albumentations -y 
# !pip uninstall folium -y 
# !pip uninstall datascience -y 
# !pip install dragonn==0.4.1 
# !pip uninstall tensorflow 
# !pip install tensorflow-gpu==1.15
# !pip install keras==2.3 


## Training on "Real" Data: SPI1 TF CHiP-seq data from ENCODE  <a name='9'>
<a href=#outline>Home</a>
We now want to use DragoNN to train and interpret a neural network on real ENCODE TF-ChIP-seq data. We will learn to predict transcription factor binding for the SPI1 transcription factor in the GM12878 cell line (one of the Tier 1 cell lines for the ENCODE project). 
    
Having done this, we want to examine how well the model is able to predict funcational SNP effects on TF binding. We compare predicted variant effect sizes from the regression models against  experimental bQTL data. The bQTL data in this way serves as a "gold-standard" validation that in silico mutagenesis on the deep learning inputs leads to correct variant effect size prediction.  We  will use bQTL data  that has been intersected with SPI1 CISBP genome motif annotations. 

In [ ]:
# SPI1, optimal IDR thresholded peaks, Myers lab, hg19, GM12878 cell type 
# https://www.encodeproject.org/experiments/ENCSR000BGQ/
!wget -O SPI1.narrowPeak.gz https://www.encodeproject.org/files/ENCFF306SRV/@@download/ENCFF306SRV.bed.gz

#Fold change bigWig track for the SPI1 dataset: 
!wget -O SPI1.pooled.fc.bigWig https://www.encodeproject.org/files/ENCFF793RKX/@@download/ENCFF793RKX.bigWig
    
## Download "ambiguous" peak sets -- these peaks are in the optimal overlap set across replicates, but are not
## found to be reproducible at a high confidence (p<0.05) by IDR. We have calculated these in advance and download 
## them from the tutorial server 
! wget -O SPI1.ambiguous.gz http://mitra.stanford.edu/kundaje/projects/dragonn/SPI1.ambiguous.gz

## Download the hg19 chromsizes file (We only use chroms 1 -22, X, Y for training)
!wget http://mitra.stanford.edu/kundaje/projects/dragonn/hg19.chrom.sizes
    
## Download the hg19 fasta reference genome (and corresponding .fai index)
!wget http://mitra.stanford.edu/kundaje/projects/dragonn/hg19.genome.fa.gz
!wget http://mitra.stanford.edu/kundaje/projects/dragonn/hg19.genome.fa.gz.fai 
!wget http://mitra.stanford.edu/kundaje/projects/dragonn/hg19.genome.fa.gz.gzi 

In [ ]:
# Download bQTL experimental data for SPI1 loci 
!wget http://mitra.stanford.edu/kundaje/projects/dragonn/SPI1.bQTLs.txt.gz

## Generating genome-wide regression labels <a name='10'>
<a href=#outline>Home</a>

We used the *genomewide_labels* function from the  [seqdataloader](https://github.com/kundajelab/seqdataloader) package to generate binned counts for the TF-ChIPseq peaks across the genome. For the sake of time, we will load the pre-generated labels in this tutorial, but the label generation code is included below if you would like to run it on your own datasets. 

seqdataloader splits the genome into 1kb regions, with a stride of 50. Each 1kb region is centered at a 200 bp bin, with a left flank of 400 bases and a right flank of 400 bases. 

* In the regression case, the asinh(mean log fold change relative to the input) in the 200 bp bin is computed. 


In [ ]:
##Download pre-generated genomewide labels 
## Regression labels 
#! wget http://mitra.stanford.edu/kundaje/projects/dragonn/SPI1.train.regression.hdf5
! wget http://mitra.stanford.edu/kundaje/projects/dragonn/cshl_2022/SPI1.train.regression.1M.hdf5
#! wget http://mitra.stanford.edu/kundaje/projects/dragonn/SPI1.valid.regression.hdf5
! wget http://mitra.stanford.edu/kundaje/projects/dragonn/SPI1.valid.regression.1M.hdf5
#! wget http://mitra.stanford.edu/kundaje/projects/dragonn/SPI1.test.regression.hdf5
! wget http://mitra.stanford.edu/kundaje/projects/dragonn/SPI1.test.regression.1M.hdf5


If you are interested in how the labels were generated, you can run the following code: 

In [ ]:
# from seqdataloader import * 
# from seqdataloader.labelgen import genomewide_labels

## seqdataloader accepts an input file, which we call SPI1.tasks.tsv, with task names in column 1, corresponding
## peak files in column 2, and the signal track in column 3. In this tutorial, the task file will have a single task entry for the SPI1 TF CHiP-seq
# with open("SPI1.task.tsv",'w') as f: 
#     f.write("task\tnarrowPeak\tbigwig\tambig\n")
#     f.write("SPI1\tSPI1.narrowPeak.gz\tSPI1.pooled.fc.bigWig\tSPI1.ambiguous.gz\n")
#f.close() 
#!cat SPI1.task.tsv

## Generate regression labels genome-wide 

##1) Training set: all chromosomes with the exception of 1,2, and 19 in our training set 

# train_set_params={
#    'task_list':"SPI1.task.tsv",
#    'outf':"SPI1.train.regression.hdf5",
#    'output_type':'hdf5',
#    'chrom_sizes':'hg19.chrom.sizes',
#    'chroms_to_exclude':['chr1','chr2','chr19','chrY'],
#    'bin_stride':50,
#    'left_flank':400,
#    'right_flank':400,
#    'bin_size':200,
#    'threads':4,
#    'subthreads':4,
#    'allow_ambiguous':True,
#    'labeling_approach':'all_genome_bins_regression'
#    }
# genomewide_labels(train_set_params)

##2) Validation set: Chromosome 1
#valid_set_params={'task_list':"SPI1.task.tsv",
#    'outf':"SPI1.valid.regression.hdf5",
#    'output_type':'hdf5',
#    'chrom_sizes':'hg19.chrom.sizes',
#    'chroms_to_keep':'chr1',
#    'bin_stride':50,
#    'left_flank':400,
#    'right_flank':400,
#    'bin_size':200,
#    'threads':1,
#    'subthreads':4,
#    'allow_ambiguous':True,
#    'labeling_approach':'all_genome_bins_regression'
#    }
#genomewide_labels(valid_set_params)

##3) Test set: Chromosomes 2, 19 
#test_set_params={
#    'task_list':"SPI1.task.tsv",
#    'outf':"SPI1.test.regression.hdf5",
#    'output_type':'hdf5',
#    'chrom_sizes':'hg19.chrom.sizes',
#    'chroms_to_keep':['chr2','chr19'],
#    'bin_stride':50,
#    'left_flank':400,
#    'right_flank':400,
#    'bin_size':200,
#    'threads':2,
#    'subthreads':4,
#    'allow_ambiguous':True,
#    'labeling_approach':'all_genome_bins_regression'
#    }
#genomewide_labels(test_set_params)


Let's examine the label files that were generated: 

In [ ]:
#The code generates bed file outputs with asinh(mean log fold change relative to the input) 
# in the central 200 bp bin 
import pandas as pd
pd.read_hdf("SPI1.train.regression.1M.hdf5",start=0,end=1000, key='data')

We will now train genome-wide regression models: 
![GenomeWideModel](https://github.com/kundajelab/dragonn/blob/master/tutorials/tutorial_images/GenomeWideModel.png?raw=1)


## Genome-wide regression model <a name='11'>
<a href=#outline>Home</a>

In [ ]:
# To prepare for model training, we import the necessary functions and submodules from keras
from keras.models import Sequential
from keras.layers import Dropout, Reshape, Dense, Activation, Flatten,Conv2D, MaxPooling2D, BatchNormalization
from keras.callbacks import EarlyStopping

In [ ]:
from dragonn.runtime_metrics import precision, recall, specificity, fpr, fnr, fdr, f1
from dragonn.custom_losses import ambig_mean_squared_error

def initialize_regression_model(ntasks=1):
    #Define the model architecture in keras (regularized, 3-layer convolution model followed by 1 dense layer)
    model=Sequential() 
    
    model.add(Conv2D(filters=50,kernel_size=(1,15),padding="same", input_shape=(1,1000,4)))
    model.add(BatchNormalization(axis=-1))
    model.add(Activation('relu'))

    model.add(Conv2D(filters=50,kernel_size=(1,15),padding="same"))
    model.add(BatchNormalization(axis=-1))
    model.add(Activation('relu'))

    model.add(Conv2D(filters=50,kernel_size=(1,13),padding="same"))
    model.add(BatchNormalization(axis=-1))
    model.add(Activation('relu'))
    
    model.add(MaxPooling2D(pool_size=(1,40)))
    
    model.add(Flatten())
    model.add(Dense(50))
    # model.add(BatchNormalization(axis=-1))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    
    model.add(Dense(ntasks))

    loss=ambig_mean_squared_error
    ##compile the model, specifying the Adam optimizer, and binary cross-entropy loss. 
    model.compile(optimizer='adam',loss=loss)
    return model

We create generators for the training and validation data: 

We upsample bins with signal greater than 0.1. 

In [ ]:
#create the generators, upsample positives to ensure they constitute 30% of each batch 
from dragonn.generators import * 

In [ ]:
#create the generators, we upsample positives to make 50% of each batch.
spi1_train_regression_gen=DataGenerator("SPI1.train.regression.1M.hdf5","hg19.genome.fa.gz", upsample_ratio=0.5, upsample_thresh=.1)
spi1_valid_regression_gen=DataGenerator("SPI1.valid.regression.1M.hdf5","hg19.genome.fa.gz",
                                        upsample=False,
                                        add_revcomp=False)

For the sake of time, we will train the model for 10 epochs, using 100 steps per epoch in training and validation.
**steps_per_epoch** indicates the number of batches that constitute a single epoch (recall that an epoch constitutes a full forward and backward pass of the dataset through the model before weights are updated). In practice, we often don't need to pass the full dataset through the model to constitute an epoch, especially when the dataset is very large. We can specify the epoch size with the **steps_per_epoch** argument. 

Use larger values when training an actual model -- a recommended epoch size is 100000 samples for training and 50000 for validation. 

In [ ]:
#Train the SPI1 regression model 
spi1_regression_model=initialize_regression_model()

## use the keras fit_generator function to train the model with early stopping after 3 epochs 
history_regression=spi1_regression_model.fit_generator(spi1_train_regression_gen,
                                                 validation_data=spi1_valid_regression_gen,
                                                 steps_per_epoch=1000,
                                                 validation_steps=200,
                                                 epochs=20,
                                                 verbose=1,
                                                 use_multiprocessing=True,
                                                 workers=20,
                                                 max_queue_size=100,
                                                 callbacks=[EarlyStopping(patience=5,restore_best_weights=True)])

In [ ]:
#import functions for visualization of data 
from dragonn.vis import *

In [ ]:
%matplotlib inline

In [ ]:
## Plot the learning curves for SPI1  
plot_learning_curve(history_regression)

The training loss curve decreases gradually over the 7 epochs of training, while the validation loss drops sharply on the first epoch and then remains relatively flat. A possible explanation is that the training set is upsampled to include 10% positives in each batch, while the validation set is not upsampled and therefor has approximately 2% positives in each batch. 

In [ ]:
from keras.models import save_model

In [ ]:
#We load a regression model that has been trained on epochs of size 100000 until the early stopping criterion
# was met. 
!wget http://mitra.stanford.edu/kundaje/projects/dragonn/cshl_2022/SPI1.regression.model.hdf5

from keras.models import load_model
from dragonn.custom_losses import * 
from dragonn.metrics import * 
custom_objects={"recall":recall,
                "sensitivity":recall,
                "specificity":specificity,
                "fpr":fpr,
                "fnr":fnr,
                "fdr":fdr,
                "precision":precision,
                "f1":f1,
                "ambig_mean_squared_error":ambig_mean_squared_error}
spi1_regression_model=load_model("SPI1.regression.model.hdf5",custom_objects=custom_objects)

We now measure how well the model performed by calculating performance metrics on the test splits across the whole genome. 

In [ ]:
#Get predictions on the test set 
spi1_test_regression_gen=DataGenerator("SPI1.test.regression.1M.hdf5",
                                       "hg19.genome.fa.gz",
                                         upsample=False,
                                         add_revcomp=False,
                                         batch_size=1000)

In [ ]:
spi1_test_regression_predictions=spi1_regression_model.predict_generator(spi1_test_regression_gen,
                                                                         max_queue_size=5000,
                                                                         workers=20,
                                                                         use_multiprocessing=True,
                                                                         verbose=1)

In [ ]:
spi1_test_regression_labels=spi1_test_regression_gen.data
spi1_test_regression_predictions=np.expand_dims(spi1_test_regression_predictions,axis=1)

In [ ]:
#remove nans, as they corresponnd to ambiguous values 
nan_indices=np.isnan(spi1_test_regression_labels.values.astype(bool))

In [ ]:
#remove nans, as they correspond to ambiguous values 
spi1_test_regression_labels=spi1_test_regression_labels[~nan_indices]
spi1_test_regression_predictions=spi1_test_regression_predictions[~nan_indices]

In [ ]:
#Calculate spearman and pearson correlation between truth labels and predictions 
from scipy.stats import pearsonr, spearmanr
corr_pearson=pearsonr(spi1_test_regression_labels['SPI1'],spi1_test_regression_predictions.ravel())
corr_spearman=spearmanr(spi1_test_regression_labels['SPI1'],spi1_test_regression_predictions.ravel())
print("Pearson correlation on test set:"+str(corr_pearson))
print("Spearman correlation on test set:"+str(corr_spearman))

## Genome-wide interpretation of true positive predictions in SPI1 <a name='13'>
<a href=#outline>Home</a>

### Regression model 

In [ ]:
#Sanity-check that the model is learning the SPI1 motif by running DeepLIFT on True Positives with high confidence 
#get the true positive predictions 
true_pos_regression=spi1_test_regression_labels[(spi1_test_regression_labels.values*spi1_test_regression_predictions)>4]
true_pos_regression.shape

In [ ]:
from dragonn.utils import one_hot_from_bed

interpretation_input_regression_spi1=one_hot_from_bed([i for i in true_pos_regression.index],"hg19.genome.fa.gz")
interpretation_input_regression_spi1.shape

In [ ]:
regression_positive_example= np.expand_dims(interpretation_input_regression_spi1[0],axis=1)

In [ ]:
#get the deeplift scoring function for regression
# deeplift_score_func_regression=get_deeplift_scoring_function("SPI1.regression.model.hdf5",target_layer_idx=-1)

In [ ]:
from dragonn.interpret import * 

regression_interpretations=multi_method_interpret(spi1_regression_model,
                                           regression_positive_example,
                                           0)

In [ ]:
plot_all_interpretations([regression_interpretations],regression_positive_example,xlim=(500,600))

If we query the sequence of the observed motif in the [TomTom](http://meme-suite.org/tools/tomtom) software from the MEME suite, we should find  that the motif is a good match for SPIB. 
An example you might observe is below: 
<img src="https://github.com/kundajelab/dragonn/blob/master/tutorials/tutorial_images/SPI1.Tut4.png?raw=1" alt="SPI12TomTom" width="400"/>

## SPI1 Binding Quantitative Trait Loci (bQTL) Data <a name='14'>
    
<a href=#outline>Home</a>

A study by Fraser et al (https://www.cell.com/fulltext/S0092-8674(16)30339-7) identified several thousand cis-acting bQTL's that affect the binding of SPI1 transcription factors in humans. We examine how effective our models are at recognizing these bQTLs. 

### Read in and annotate the bQTL dataset <a name='a'>
<a href=#outline>Home</a>


In [ ]:
## Download the bQTL dataset 
!wget http://mitra.stanford.edu/kundaje/projects/dragonn/SPI1.bQTLs.txt.gz

In [ ]:
#Read in the bQTL dataframe
bqtls=pd.read_csv("SPI1.bQTLs.txt.gz",header=0,sep='\t')
bqtls.head

In [ ]:
#Calculate the number of bQTL's
print(bqtls.shape)
n_bqtls=bqtls.shape[0]

In [ ]:
#sort the bqtl's by p-value 
bqtls=bqtls.sort_values(by=['pvalue'])

We now annotate the bQTL's with the false discovery rate, the mean fold change bigWig signal in the bQTL region, and the logratio fo the post-CHIP frequency relative to the pre-CHIP frequency. Some of these operations are time-consuming, particularly the calculation of the mean fold change bigWig signal. Hence,we have pre-run them. If you would like to execute these operations, uncomment the code cells below. 

In [ ]:
!wget http://mitra.stanford.edu/kundaje/projects/dragonn/SPI1.bQTLs.additional.metrics.txt.gz
bqtls=pd.read_csv("SPI1.bQTLs.additional.metrics.txt.gz",header=0,sep='\t')
bqtls.head

In [ ]:
##Calculate the Benjamini-Hochberg FDR 
#bqtls['FDR']=1.0
#for index,row in bqtls.iterrows(): 
#    row['FDR']=min(row['FDR'],row['pvalue']*n_bqtls/(index+1))

In [ ]:
## Calculate logratio of the Post-CHIP frequency to the pre-CHIP frequency
#bqtls['logratio']=np.log((bqtls['POSTfreq']+.01)/(bqtls['prechipfreq']+0.01))

In [ ]:
## IMPORTANT! Convert the bQTL coordinates from 1-indexing to 0-indexing
#bqtls['start']=bqtls['position']-1
#bqtls['end']=bqtls['position']

In [ ]:
## Calculate the mean fold change bigWig signal within the 200 bp region 
## centered on the bQTL

##initialize the field within the Pandas dataframe with default value of 0
#bqtls['mean_chipseq_fc']=0

##use the pyBigWig library to calculate the mean fold change bigWig signal 
##within the 200 bp region centered at the bQTL 
#import pyBigWig 
#bigwig_fh = pyBigWig.open("SPI1.pooled.fc.bigWig")
#for index,row in bqtls.iterrows():
#    values = bigwig_fh.values(row['Chr'],
#                              row['end']-100,
#                              row['end']+100,
#                              numpy=True)
#    row['mean_chipseq_fc'] = np.mean(values)
#    if (index%1000 == 0):
#        print("Done",index)

In [ ]:
##Let's save the augmented bQTL data frame so we can resume from 
## this point in the analysis more easily. 
#bqtls.to_csv("SPI1.bQTLs.additional.metrics.txt.gz",
#            sep='\t',
#            header=True,
#            index=False,
#            compression='gzip')

### DNN predictions for Post and Alt alleles within the bQTL dataset  <a name='b'>
<a href=#outline>Home</a>

We would like to obtain  predictions for the reference and alternate alleles at each bQTL. Because we will visualize the difference in predictions for the reference and alternate bQTL alleles, we will operate in logit space -- the layer in the model whose output feeds into the final sigmoid layer. We do this because calculating the logratio of model predictions in logit space is more correct than subtracting probabilities from the model's final sigmoid output layer. 

Consequently, we obtain the predictions of the regression model.

In [ ]:
# get model predictions for bQTLs 
from dragonn.generators import * 
bqtl_post_gen=BQTLGenerator("SPI1.bQTLs.txt.gz","hg19.genome.fa.gz","POSTallele")
bqtl_alt_gen=BQTLGenerator("SPI1.bQTLs.txt.gz","hg19.genome.fa.gz","ALTallele")

In [ ]:
bqtl_post_regression_predictions=spi1_regression_model.predict_generator(bqtl_post_gen,
                                                                          max_queue_size=5000,
                                                                          workers=40,
                                                                          use_multiprocessing=True,
                                                                          verbose=1)

In [ ]:
bqtl_alt_regression_predictions=spi1_regression_model.predict_generator(bqtl_alt_gen,
                                                                          max_queue_size=5000,
                                                                          workers=40,
                                                                          use_multiprocessing=True,
                                                                          verbose=1)

Let's augment the bQTL dataframe with the model predictions. Let's summarize the predictions we have generated on the bQTL dataset: 

* Regression model predictions for the POST allele 
* Regression model predictions for the ALT allele 



In [ ]:
bqtls['post_regression_predictions']=bqtl_post_regression_predictions
bqtls['alt_regression_predictions']=bqtl_alt_regression_predictions

## Compare model predictions on POST and ALT alleles  <a name='c'>
<a href=#outline>Home</a>


We now assess whether the model predictions of accessibility change for significant bQTL's when the reference allele is replaced with the alternate allele in the input sequence. We also compare the distribution of reference vs alternate predictions for the set of significant bQTLS to the corresponding distribution for the set of non-significant bQTLs. 

First, we must select comparable subsets of non-significant and significant bQTLs from the bQTL dataset. For our purposes, the sets are comparable if they have similar distributions of accessibility predictions, where predictions are defined as the max of the reference and alternate allele predictions. To ensure that we are comparing such "matched" subsets, we perform the following steps:  

In [ ]:
#We define a temporary field to store the maximum of the 
# regression predictions for the alternate and reference alleles 
bqtls['max_regression_predictions']=bqtls[['post_regression_predictions',
                                          'alt_regression_predictions']].max(axis=1)


In [ ]:
significant_bqtls=bqtls[bqtls['pvalue']<=5e-5]
non_significant_bqtls=bqtls[bqtls['pvalue']==1]

In [ ]:
significant_bqtls.shape

In [ ]:
non_significant_bqtls.shape

We define a helper function to sample bQTLS from the bQTL dataset to match another subset of bQTLS according to a user-specified attr_name. This function will allow us to compare the accessibility predictions for significant bQTLS with the non-significant bQTLS. 

In [ ]:
from plotnine import * 
def plot_distribution(bqtls_to_match,matched_sampled_bqtls,attr_name):
    toplot=pd.DataFrame({"bqtls_to_match":bqtls_to_match[attr_name],
                        "matched_sampled_bqtls":matched_sampled_bqtls[attr_name]})
    toplot=toplot.melt()
    print((ggplot(toplot,aes(x="value",group="variable",color="variable",fill="variable"))
     +geom_density(alpha=0.3)
     +theme_bw()
     +xlab(attr_name)
     +ylab("Density")))

    print((ggplot(bqtls_to_match,aes(x=attr_name))
     +geom_density(alpha=0.3)
     +theme_bw()
     +xlab(attr_name)
     +ylab("Density")
     +ggtitle("bqtls_to_match")))

    print((ggplot(matched_sampled_bqtls,aes(x=attr_name))
     +geom_density(alpha=0.3)
     +theme_bw()
     +xlab(attr_name)
     +ylab("Density")
     +ggtitle("matched_sampled_bqtls")))



def sample_matched_bqtls(bqtls_to_match, bqtls_to_sample, attr_name):
    #sort bqtls_to_sample by attr_name
    sorted_bqtls_to_sample=bqtls_to_sample.sort_values(by=[attr_name])
    sorted_bqtls_to_sample_vals = sorted_bqtls_to_sample[attr_name]
    
    bqtls_to_match_vals = bqtls_to_match[attr_name]
    
    #find indices in the bqtls_to_match_vals Series that are close in value to corresponding entries 
    # from sorted_bqtls_to_sample_vals
    searchsorted_indices = np.searchsorted(a=np.array(sorted_bqtls_to_sample_vals), 
                                           v=np.array(bqtls_to_match_vals))
    
    matched_sampled_bqtls_indices = set()
    
    for idx in searchsorted_indices:
        #shift the index until you find one that isn't taken
        shift = 1
        while (idx in matched_sampled_bqtls_indices or idx==len(sorted_bqtls_to_sample)):
            if idx == len(sorted_bqtls_to_sample):
                shift = -1
            idx += shift
        if (idx < 0 or idx > len(sorted_bqtls_to_sample)):
            print(idx)
        matched_sampled_bqtls_indices.add(idx)
    
    matched_sampled_bqtls = sorted_bqtls_to_sample.iloc[list(matched_sampled_bqtls_indices)]
    
    plot_distribution(bqtls_to_match,matched_sampled_bqtls,attr_name)
        
    return matched_sampled_bqtls

We now utilize the "sample_matched_bqtls" helper function to select matched subsets of significant and non-significant bQTLs by model prediction values 

In [ ]:
matched_bqtls_maxaltpost = sample_matched_bqtls(bqtls_to_match=significant_bqtls,
                                                     bqtls_to_sample=non_significant_bqtls,
                                                     attr_name="max_regression_predictions")

We have now verified that the distribution of max(alt allele prediction, ref allele predictions) for the significant bQTL subset matches the corresponding distribution for the non-significant bQTL subset. 

Our next step is to generate a scatterplot examining the model predictions for reference vs alternate alleles within the matched significant and non-significant subsets: 

In [ ]:
#concatenate the subset of significant bQTL's and the matched subset of non-significant bQTL's for visualization on the 
# same plotting axes 
significant_bqtls['Significant_bQTL']=True
matched_bqtls_maxaltpost['Significant_bQTL']=False 
to_score_bqtls=pd.concat([significant_bqtls,matched_bqtls_maxaltpost],axis=0)

In [ ]:
#concatenate the subset of significant bQTL's and the matched subset of non-significant bQTL's for visualization on the 
# same plotting axes 
significant_bqtls['Significant_bQTL']=True
matched_bqtls_maxaltpost['Significant_bQTL']=False 
to_score_bqtls=pd.concat([significant_bqtls,matched_bqtls_maxaltpost],axis=0)

In [ ]:
significant_to_score_bqtls=to_score_bqtls[to_score_bqtls['Significant_bQTL']==True]
insignificant_to_score_bqtls=to_score_bqtls[to_score_bqtls['Significant_bQTL']==False]


In [ ]:
#use the plotnine plotting library to generate a scatterplot of  predictions 
# for the POST and ALT alleles within the significant and matched non-significant bQTL subsets. 


print((ggplot(significant_to_score_bqtls,aes(x="post_regression_predictions",
                y="alt_regression_predictions"))
             +geom_point(alpha=0.3,color="#0000FF")
             +xlab("POST regression predictions")
             +ylab("ALT regression predictions")
             +theme_bw()
             +ggtitle("Significant bQTLS")))

print((ggplot(insignificant_to_score_bqtls,aes(x="post_regression_predictions",
                y="alt_regression_predictions"))
             +geom_point(alpha=0.3,color="#FF0000")
             +xlab("POST regression predictions")
             +ylab("ALT regression predictions")
             +theme_bw()
             +ggtitle("Non-Significant bQTLS")))


As expected, we observe a much higher fraction of significant bQTL's deviating from the diagonal of the plot relative to the non-significant bQTL's. This makes sense -- if a bQTL lies along the diagonal, then the model prediction for the POST allele is very close to the prediction for the ALT allele (i.e. the SNP does not have a strong effect on prediction of accessibility). However, when a bQTL lies far from the diagonal, the model predicts a different probability of chromatin accessibility for the POST and ALT alleles -- i.e. the SNP is disrupting TF binding. 

## bQTL dataset motif scan with HOCOMOCO SPI1 motif  <a name='d'>
<a href=#outline>Home</a>

Our next step is to annotate the to-score bQTLs with the fasta sequence around them and do motif scoring to determine whether the SPI1 motif is driving the change in model predictions for the reference and altenate alleles.

In [ ]:
#We save the "to_score" bQTL subset to an output file so we can reproduce this analysis in the future starting from the 
# motif scoring step. 
to_score_bqtls.to_csv("SPI1.bQTLs.toScore.txt.gz",
                      sep='\t',
                      index=False,
                      compression="gzip")
num_to_score=to_score_bqtls.shape[0]

In [ ]:
#We use a flank size of 10 on either side of the bQTL 
flank_size=10

#Once again, we initialize our POST and ALT generators for the bQTL dataset, focusing only on the significant bQTLs and 
# our matched non-significant subset. 
#We use a hack where we set the batch size equal to the number of bQTL's to be scored so we get the one-hot-encoding 
#for all bQTL's simultaneously in one batch 
bqtl_post_gen=BQTLGenerator("SPI1.bQTLs.toScore.txt.gz",
                            "hg19.genome.fa.gz",
                            "POSTallele",
                            flank_size=flank_size,
                            batch_size=num_to_score)
bqtl_alt_gen=BQTLGenerator("SPI1.bQTLs.toScore.txt.gz",
                           "hg19.genome.fa.gz",
                           "ALTallele",
                           flank_size=flank_size,
                           batch_size=num_to_score)



We now load the SPI1 count matrix from the [HOCOMOCO](http://hocomoco11.autosome.ru/motif/SPI1_HUMAN.H11MO.0.A) database. 

In [ ]:
! [[ -e SPI1.pcm ]] || curl https://hocomoco11.autosome.org/final_bundle/hocomoco11/full/HUMAN/mono/pcm/SPI1_HUMAN.H11MO.0.A.pcm > SPI1.pcm

In [ ]:
! cat SPI1.pcm

In [ ]:
spi1_pcm = np.array([
                [float(x) for x in row.rstrip().split("\t")]
                for (i,row) in enumerate(open("SPI1.pcm")) if i > 0]).transpose((1,0))

In [ ]:
spi1_pcm

In [ ]:
#add a pseudocount for motif normalization 
pseudocount=10e-4
spi1_pcm+=pseudocount
spi1_pcm

In [ ]:
#calculate the frequency matrix from the count matrix 
#print row-sums 
np.sum(spi1_pcm,axis=0)
#normalize by row-sums 
spi1_pfm=spi1_pcm/np.sum(spi1_pcm,axis=0)
#sanity-check that the row-sums for the position frequency matrix (pfm) are 1 
np.sum(spi1_pfm,axis=0)

In [ ]:
#plot the pwm 
plot_bases(spi1_pfm.transpose(),figsize=(6,3))

In [ ]:
#plot the reverse complement of the pwm 
from dragonn.utils import reverse_complement
plot_bases(reverse_complement(spi1_pfm.transpose()),figsize=(6,3))

Note that the high confidence true positive SPI1 peaks we interpreted have deepLIFT score profiles resembling the 
reverse complement SPI1B motif. 

For each SNP in our bQTL scoring subset, we compute the maximum motif scan score along the 20 bp flanking the reference allele and the alternate allele. In this motif scan, we consider both the motif logo and the reverse complement of the motif logo. We then calculate the difference in the maximum motif score for the alternate allele and the reference allele. These scores should be higher for the significant subset of bQTLs compared to the non-significant subset of bQTLs. We verify that this is indeed the case. 

In peforming the motif scan, we normalize relative to the gc content of the SPI1 peaks in our dataset. Let's calculate this value.

In [ ]:
# We should have the SPI1.narrowPeak.gz file already downloaded, but just in case  here is the link to download 
# it again. 
#!wget http://mitra.stanford.edu/kundaje/projects/dragonn/SPI1.narrowPeak.gz
from dragonn.utils import allele_freqs_from_bed, get_motif_scores
spi1_peak_freqs=allele_freqs_from_bed("SPI1.narrowPeak.gz","hg19.genome.fa.gz")

In [ ]:
# a, c, g, t frequqencies 
print(spi1_peak_freqs)
GC_fraction=spi1_peak_freqs[1]+spi1_peak_freqs[2]
print(GC_fraction)

In [ ]:
#Get the maximum sequence SPI1B scan scores for the reference allele 
to_score_bqtls['scan_post_scores']=np.max(
    get_motif_scores(
        bqtl_post_gen[0],
        ["SPI1B"],
        GC_fraction=GC_fraction,
        pfm=spi1_pfm,
        include_rc=True)
    ,axis=2).squeeze()

In [ ]:
#Get the maximum sequence SPI1B scan scores for the alternate alelle 
to_score_bqtls['scan_alt_scores']=np.max(
    get_motif_scores(
        bqtl_alt_gen[0],
        ["SPI1B"],
        GC_fraction=GC_fraction,
        pfm=spi1_pfm, 
        include_rc=True)
    ,axis=2).squeeze()

In [ ]:
#Compute the scan score delta for the reference allele - alternate allele 
to_score_bqtls['scan_delta']=to_score_bqtls['scan_post_scores']-to_score_bqtls['scan_alt_scores']

We summarize our findings below: 

### bQTL interpretation summary: deep learning models vs motif scan  <a name='e'>
<a href=#outline>Home</a>

#### Motif scan 

In [ ]:
#We plot the motif scan scores for the POST and ALT alleles: 
significant_to_score_bqtls=to_score_bqtls[to_score_bqtls['Significant_bQTL']==True]
insignificant_to_score_bqtls=to_score_bqtls[to_score_bqtls['Significant_bQTL']==False]

print((ggplot(significant_to_score_bqtls,aes(x="scan_post_scores",
                y="scan_alt_scores"))
             +geom_point(alpha=0.3,color="#0000FF")
             +xlab("POST allele motif scan score for SPI1B")
             +ylab("ALT allele motif scan score for SPI1B")
             +ggtitle("Significant bQTLs")
             +theme_bw()))
print((ggplot(insignificant_to_score_bqtls,aes(x="scan_post_scores",
                y="scan_alt_scores"))
             +geom_point(alpha=0.3,color="#FF0000")
             +xlab("POST allele motif scan score for SPI1B")
             +ylab("ALT allele motif scan score for SPI1B")
             +ggtitle("Non-Significant bQTLs")
             +theme_bw()))


Recall that we defined the bQTL logratio as the  logratio of the Post-CHIP frequency to the pre-CHIP frequency. Plotting the bQTL logratio vs the delta of the PWM scan score gives: 


In [ ]:
print((ggplot(to_score_bqtls,aes(x="logratio",
                y="scan_delta",
                group="Significant_bQTL",
                color="Significant_bQTL"))
             +geom_point(alpha=0.3)
             +xlab("bQTL logratio")
             +ylab("POST allele motif scan score - ALT allele motif scan")
             +theme_bw()))


#### Model predictions 

In [ ]:
#We plot the model predictions for the POST and ALT alleles
# (we examined this plot above, it is reproduced here for context with the other comparisons)

print((ggplot(significant_to_score_bqtls,aes(x="post_regression_predictions",
                y="alt_regression_predictions"))
             +geom_point(alpha=0.3,color="#0000FF")
             +xlab("POST allele regression model prediction")
             +ylab("ALT allele regression model prediction")
             +ggtitle("Signicant bQTLs")
             +theme_bw()))
print((ggplot(insignificant_to_score_bqtls,aes(x="post_regression_predictions",
                y="alt_regression_predictions"))
             +geom_point(alpha=0.3,color="#FF0000")
             +xlab("POST allele regression model prediction")
             +ylab("ALT allele regression model prediction")
             +ggtitle("Non-Signicant bQTLs")
             +theme_bw()))

In [ ]:
#We plot the logratio for the bQTLs vs the delta in logit space 

#we extract the fields we need for plotting
to_plot=pd.DataFrame({'Significant_bQTL':to_score_bqtls['Significant_bQTL'],
                      'logratio':to_score_bqtls['logratio'],
                     'delta_regression_predictions':
                      to_score_bqtls['post_regression_predictions']-
                      to_score_bqtls['alt_regression_predictions']})

print((ggplot(to_plot,aes(x="logratio",
                y="delta_regression_predictions",
                color="Significant_bQTL",
                fill="Significant_bQTL",
                group="Significant_bQTL"))
             +geom_point(alpha=0.3)
             +xlab("bQTL logratio")
             +ylab("POST allele regression prediction - \nALT allele regression prediction")
             +theme_bw()))


Finally, we restrict our analysis to the subset of bQTL's where the model's prediction changed most dramatically betwen the POST and ALT alleles. For this purpose, we subset the to_score_bqtls to those bQTL's where : 

* POST prediction > 0.9 and ALT prediction < 0.5
or 
* POST prediction < 0.5 and ALT prediction > 0.9 

In [ ]:
condition1=(to_score_bqtls['post_regression_predictions']>0.9) & (to_score_bqtls['alt_regression_predictions']<0.5)
condition2=(to_score_bqtls['post_regression_predictions']<0.5) & (to_score_bqtls['alt_regression_predictions']>0.9)

to_keep=condition1 | condition2 
confident_to_score_bqtls=to_score_bqtls[to_keep]

In [ ]:
confident_to_score_bqtls.shape

In [ ]:
#We plot the model predictions for the POST and ALT alleles
# (we examined this plot above, it is reproduced here for context with the other comparisons)

print((ggplot(confident_to_score_bqtls,aes(x="post_regression_predictions",
                y="alt_regression_predictions",
                group="Significant_bQTL",
                color="Significant_bQTL"))
             +geom_point(alpha=0.5)
             +xlab("POST allele regression model prediction")
             +ylab("ALT allele regression model prediction")
             +scale_color_manual(values=["#FF0000","#0000FF"])
             +theme_bw()))

In [ ]:
#We plot the logratio for the bQTLs vs the delta in predictions 

#we extract the fields we need for plotting
to_plot=pd.DataFrame({'Significant_bQTL':confident_to_score_bqtls['Significant_bQTL'],
                      'logratio':confident_to_score_bqtls['logratio'],
                     'delta_regression_predictions':
                      confident_to_score_bqtls['post_regression_predictions']-
                      confident_to_score_bqtls['alt_regression_predictions']})

print((ggplot(to_plot,aes(x="logratio",
                y="delta_regression_predictions",
                color="Significant_bQTL",
                fill="Significant_bQTL",
                group="Significant_bQTL"))
             +geom_point(alpha=0.3)
             +xlab("bQTL logratio")
             +ylab("POST allele regression logit  - \nALT allele regression logit")
             +scale_color_manual(values=["#FF0000","#0000FF"])
             +theme_bw()))

Our visualizations of the logratio values vs the delta in motif scan scores and model predictions are quite informative. For the significant bQTL's, we see a positive correlation between logratio and delta in model predictions. However, this correlation is nearly non-existent for the motif scan, suggesting the higher sensitivity of the deep learning model. Let's test this out on a low-affinity transcription factor example: 


## Deep learning models are able to identify low-affinity TF-binding sites missed by motif scanning. <a name='15'>
<a href=#outline>Home</a>

Let's examine the region **chr5:107857257:107857288**. We have a highly significant bQTL within that region: 

In [ ]:
to_score_bqtls[(to_score_bqtls['Chr']=="chr5") & (to_score_bqtls['position']>107857256) & (to_score_bqtls['position']<107857288)]

Let's extract the one-hot-encoded 1kb region centered at this position and 

1) interpret the region with our deep learning models

2) scan the region with the canonical SPI1 motif

We can compare how the two approaches perform. 

In [ ]:
bqtl_post_gen=BQTLGenerator("SPI1.bQTLs.toScore.txt.gz",
                            "hg19.genome.fa.gz",
                            "POSTallele",
                            batch_size=num_to_score)
bqtl_alt_gen=BQTLGenerator("SPI1.bQTLs.toScore.txt.gz",
                           "hg19.genome.fa.gz",
                           "ALTallele",
                           batch_size=num_to_score)



In [ ]:
post_seq=np.expand_dims(bqtl_post_gen[0][1622],axis=0)
alt_seq=np.expand_dims(bqtl_alt_gen[0][1622],axis=0)

In [ ]:
post_seq_regression_interpretations=multi_method_interpret(spi1_regression_model,
                                           post_seq,
                                           0,
                                           motif_names=['SPI1B'],
                                           pfm=spi1_pfm,
                                           GC_fraction=GC_fraction)

In [ ]:
plot_all_interpretations([post_seq_regression_interpretations],post_seq,xlim=(450,550),snp_pos=501)

DeepLIFT identifies the region centered at the bQTL as important for predicting accessiblity, and the deepLIFT track matches the PWM for SPI1B quite well. The one exception is position 12 in the PWM -- our sequence has a "T" at that 
position, which both the model and the PWM strongly disfavor. However, the model is sensitive enough to recognize that the region is still a weak-affinity SPI1B site, whereas the motif scan lacks that sensitivity -- the motif scan track returns non-significant scores throughout the 1kb region. 


Although our motif scan gave random-looking scores, let's see how our scan results compare relative to the "best possible" motif hit, the "worst possible" motif hit, and an "average" region selected from the genome. 

In [ ]:
#identify indices of the bases with highest probability at each position in the PFM
best_match_bases=np.argmax(spi1_pfm,axis=0)
worst_match_bases=np.argmin(spi1_pfm,axis=0)
mask=np.zeros(spi1_pfm.shape)

best_hits=mask.copy() 
worst_hits=mask.copy() 

#generate one-hot encoding of the best and worst hits 
for index in range(mask.shape[1]): 
    cur_best_base=best_match_bases[index]
    cur_worst_base=worst_match_bases[index]
    best_hits[cur_best_base,index]=1
    worst_hits[cur_worst_base,index]=1

#generate a PFM consistent with the genome average base frequencies 
genome_background=np.tile(np.array([0.26,0.23,0.23,0.26]),(spi1_pfm.shape[1],1)).transpose()

#transpose the PFM's and expand dimensions to get a format compatible for motif scanning
best_hits=np.expand_dims(np.expand_dims(np.transpose(best_hits),axis=0),axis=0)
worst_hits=np.expand_dims(np.expand_dims(np.transpose(worst_hits),axis=0),axis=0)
genome_background=np.expand_dims(np.expand_dims(np.transpose(genome_background),axis=0),axis=0)


In [ ]:
best_score=np.max(get_motif_scores(best_hits,['SPI1'],GC_fraction=GC_fraction,pfm=spi1_pfm,include_rc=False))
print(best_score)

In [ ]:
worst_score=np.min(get_motif_scores(worst_hits,['SPI1'],GC_fraction=GC_fraction,pfm=spi1_pfm,include_rc=False))
print(worst_score)

In [ ]:
genome_background_score=np.max(get_motif_scores(genome_background,['SPI1'],GC_fraction=GC_fraction,pfm=spi1_pfm,include_rc=False))
print(genome_background_score)

How does this compare to our observed score? 

In [ ]:
post_score=get_motif_scores(post_seq,['SPI1'],GC_fraction=GC_fraction,pfm=spi1_pfm,include_rc=True)
print(np.max(post_score))


We overlay the best possible motif match, the worst possible motif match, the observed motif score track, and the genome background score track to illustrate that motif scanning does not effectively identify the SPI1 hit. 

In [ ]:
f,axes=plt.subplots(1,dpi=80,figsize=(20,6))
axes.plot(post_score.squeeze(), "-ko",label="Observed motif scores at low affinity site (1kb region)")
axes.axhline(y=worst_score,color="r",label="Worst possible SPI1 score")
axes.axhline(y=best_score,color="b",label="Best possible SPI1 score")
axes.axhline(y=genome_background_score,color="green",label="Mean genome background  SPI1 score")
axes.axhline(y=np.max(post_score),color="orange",label="Max of observed SPI1 scores at low affinity site ")
axes.set_xlabel("Sequence base")
plt.legend()
plt.show()

In summary, we conclude that the low-affinity SPI1 motif would have been missed by a classical PWM-scanning approach, but is clearly idenfitied via DeepLIFT and ISM analysis on the regression models. 